# Solving the Diffusion Equation in Julia #

This notebook revises the following example that solves the Diffusion equation in Julia using Finite Difference Methods.  

https://github.com/JuliaDiffEq/DiffEqOperators.jl/blob/master/docs/HeatEquation.md

We change the syntax slightly to facilitate modifying the problem but also show an animation and Hovmoller plot to depict the entire solution.

In [1]:
# Import packages

using DiffEqOperators, DifferentialEquations, Plots, LaTeXStrings

The Diffusion equation in the case of possibly non-uniform diffusion can be written as,

$$\frac{\partial u}{\partial t} = \frac{{\partial}}{\partial x} \left( \kappa(x) \frac{\partial u}{\partial x} \right)  $$

To solve this problem numerically requires two things: discretizing the equation in space and discretizing the equation in time.  This requires we

1. discretize space using DerivativeOperator
2. discretize time using solve

Both of these are part of the DiffEqOperators Package.

In [70]:
# Define the spatial grid

N = 64;                              # Number of spatial grid points
L = 5.0;                              # Length of domain
Δx = 2*L/(N-1);                      # spatial grid scale
x = collect(-L : Δx : L);            # spatial domain

#boundary = "periodic";
boundary = "Dirichlet";
#boundary = "Neumann";

In [71]:
# Define the temporal grid

Δt = 0.1;                            # Temporal resolution
tI = 0.0;                            # Initial time
tF = 10.0;                           # Final time
t  = collect(tI:Δt:tF);              # temporal grid

In [72]:
# Define differentiation matrix

ord = 2;               # Order of approximation

if boundary=="periodic"
    display("Building matrix using periodic boundary conditions.")
    D1  = DerivativeOperator{Float64}(1,ord,Δx,N,:periodic,:periodic);    # 1nd Derivative
    D2  = DerivativeOperator{Float64}(2,ord,Δx,N,:periodic,:periodic);    # 2nd Derivative
elseif boundary=="Dirichlet"
    display("Building matrux using Dirichlet boundary conditions")
    D1  = DerivativeOperator{Float64}(1,ord,Δx,N,:Dirichlet0,:Dirichlet0);    # 1nd Derivative
    D2  = DerivativeOperator{Float64}(2,ord,Δx,N,:Dirichlet0,:Dirichlet0);    # 2nd Derivative
elseif boundary=="Neumann"
    display("Building matrix using Neumann boundary conditions")
    D1  = DerivativeOperator{Float64}(1,ord,Δx,N,:Neumann0,:Neumann0);    # 1nd Derivative
    D2  = DerivativeOperator{Float64}(2,ord,Δx,N,:Neumann0,:Neumann0);    # 2nd Derivative
end

"Building matrux using Dirichlet boundary conditions"

DiffEqOperators.DerivativeOperator{Float64,SVector{3,Float64},:Dirichlet0,:Dirichlet0}(2, 2, 0.15873015873015872, 64, 3, [1.0, -2.0, 1.0], (3, 3), (4, 4), Base.RefValue{Array{Array{Float64,1},1}}(Array{Float64,1}[]), Base.RefValue{Array{Array{Float64,1},1}}(Array{Float64,1}[]), Base.RefValue{Tuple{Tuple{Float64,Float64,Any},Tuple{Float64,Float64,Any}}}(((1.0, 0.0, 0.0), (1.0, 0.0, 0.0))), Base.RefValue{Int64}(0))

In [73]:
# Plot initial conditions

u0  = exp.(-x.^2); #-(x - 0.5).^2 + 1/12;   
du1 = D1*u0
du2 = D2*u0
plot(x,   u0, lw=3, xlab="x", title="Initial Condition", label="u0")
plot!(x, du1, lw=3, label="du0/dx")
plot!(x, du2, lw=3, label="d2u0/dx2")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -5.0 
 
 
 -2.5 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 -1.5 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 Initial Condition 
 
 
 x 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 u0 
 
 
 
 du0/dx 
 
 
 
 d2u0/dx2

In [74]:
# Define function to specify RHS of PDE

κ(x) = 1.0 + 0.0*x
const tmp1 = similar(x)
const tmp2 = similar(x)
function f(t,u,du)
    A_mul_B!(tmp1,  D1, u)
    @. tmp2 = κ(x)*tmp1
    A_mul_B!(du,D1,tmp2)
end

f (generic function with 1 method)

In [75]:
prob1 = ODEProblem(f, u0, (tI,tF));               # Set up ODE
sol1 = solve(prob1, dense=false, saveat=0.1);     # Solve ODE

In [76]:
# Plot the solution at the first 10 time steps (change???)

plot(x, [sol1(i) for i in 0:1:10], lw=3)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -5.0 
 
 
 -2.5 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1 
 
 
 
 y2 
 
 
 
 y3 
 
 
 
 y4 
 
 
 
 y5 
 
 
 
 y6 
 
 
 
 y7 
 
 
 
 y8 
 
 
 
 y9 
 
 
 
 y10 
 
 
 
 y11

In [48]:
# Create an animation of the solution

@gif for i in 1:length(sol1)
    plot(x,sol1[i], xlims=(-4,4), ylims=(0,1.2), xlabel = "x", ylabel = "u(x)", title="Diffusion Equation Simulation",legend=false,lw=3)

    end every 2

INFO: Saved animation to /home/fpoulin/Documents/Teaching/2018/AMATH353/JuliaNotebooks/tmp.gif


Plots.AnimatedGif("/home/fpoulin/Documents/Teaching/2018/AMATH353/JuliaNotebooks/tmp.gif")

In [49]:
# Plot a Hovmoller Plot

contourf(x,t,convert(Array,sol1)', xlab="space", ylab="time", title="Hovmoller Plot")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -5.0 
 
 
 -2.5 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 Hovmoller Plot 
 
 
 space 
 
 
 time 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0.4 
 
 
 0.5 
 
 
 0.6 
 
 
 0.7 
 
 
 0.8 
 
 
 0.9 
 
<polyline clip-path="url(#clip5700)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" points="
 550.315,360.065 550.315,327.001 556.315,327.001 550.315,327.001 550.315,293.936 556.315,293.936 550.315,293.936 550.315,260.872 556.315,260.872 550.315,260.872 
 550.315,227.807 556.315,227.807 550.315,227.807 550.315,194.743 556.315,194.743 550.315,194.743 550.315,161.678 556.315,161.678 550.315,161.678 550.315,128.614 
 556.315,128.614 550.315,128.614 550.315,95.549 556.315,95.549 550.315,95.549 550.315,62.4845 556.315,62.4845 550.315,62.4845 550.315,31.4961 
 "/>

In [23]:
using DiffEqOperators

Δx = .025π
#N = Int(2*(π/Δx)) -2
N = round(Int,2*(π/Δx)) -2
ord= N              # Order of approximation
x = -π:Δx:π-Δx
D1  = DerivativeOperator{Float64}(1,ord,Δx,N,:periodic,:periodic);    # 1nd Derivative
D2  = DerivativeOperator{Float64}(2,ord,Δx,N,:periodic,:periodic);    # 2nd Derivative

u0 = cos.(x)
du_true = -cos.(x)

M = full(Tridiagonal([1.0 for i in 1:N+1],[-2.0 for i in 1:N+2],[1.0 for i in 1:N+1]))
# Do the reflections, different for x and y operators
M[end,1] = 1.0
M[1,end] = 1.0
M = M/(Δx^2)
M*u0 ≈ D2*u0

A = zeros(M)
for i in 1:N+2, j in 1:N+2
    i == j && (A[i,j] = -0.5)
    abs(i - j) == 2 && (A[i,j] = 0.25)
end
A[end-1,1] = 0.25
A[end,2] = 0.25
A[1,end-1] = 0.25
A[2,end] = 0.25
A = A/(Δx^2)

A*u0 ≈ D1*(D1*u0)

κ(x) = 1.0
const tmp1 = similar(x)
const tmp2 = similar(x)
function f(t,u,du)
    A_mul_B!(tmp1,  D1, u)
    @. tmp2 = κ(x)*tmp1
    A_mul_B!(du,D1,tmp2)
end

du = similar(u0)
f(0,u0,du)
du ≈ A*u0

using Plots; plotly()
error = du_true - du
du2 = D2*u0
error2 = du_true - du
plot(x,error)
maximum(error)
#plot(x,error2)
#plot(x,-cos.(x))
#plot(x,du)
#plot(x,cos.(x))

5.062616992290714e-14